In [ ]:
import csv
import os
import pandas as pd
import re
import time
from datetime import datetime
from pathlib import *
import random
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from pyvirtualdisplay import Display
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, \
    WebDriverException, StaleElementReferenceException, ElementNotInteractableException,\
    InvalidArgumentException, UnexpectedAlertPresentException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import socket
import time
from ping3 import ping, verbose_ping
import pytz


# Start a virtual display
display = Display(visible=0, size=(1920, 1080))
display.start()


# Define the Vietnam timezone
local_tz = pytz.timezone('Asia/Ho_Chi_Minh')
# Get the current time in UTC
now_utc = datetime.now(pytz.utc)
# Convert UTC time to local time
now = now_utc.astimezone(local_tz).date().strftime('%Y-%m-%d')
now_1 = now_utc.astimezone(local_tz).strftime('%Y-%m-%d')


class LABOR:
    def __init__(self, list_link: list, start_link: str, list_province):
        self.list_link = list_link
        self.start_link = start_link
        self.list_province_temp = None
        self.driver = self.chrome_drive()
        self.wait = WebDriverWait(self.driver, 50)
        self.action = ActionChains(self.driver)
        self.Brand = "Apple"
        self.file_name = f"../a_output/tz-apple_{now}.csv"
        for link in self.list_link[list_link.index(start_link):]:
            print(link)
            self.list_province_temp = None  # At the beginning of every link, set it none
            self.restart_province = None  # At the beginning of every link, set it none
            self.check(link, list_province_to_check=list_province)
        self.driver.quit()


    def chrome_drive(self):
    
        # Configure Chrome options
        options = webdriver.ChromeOptions()
        # options.add_argument('--headless')  # Run in headless mode
        options.add_argument('--disable-gpu')  # Disable GPU hardware acceleration
        options.add_argument('--no-sandbox')  # Bypass OS security model
        options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
    
        # Create a driver instance
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    
        # Disable the "Chrome is being controlled by automated test software" notification
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
        # # Disable the "navigator.webdriver" property
        options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    
        # Disable the "Chrome is being controlled by automated test software" banner
        options.add_argument("--disable-blink-features=AutomationControlled")
    
        # Maximize the window to avoid fingerprinting based on screen resolution
        options.add_argument("start-maximized")
    
        # Disabling the Automation Extension can help prevent detection as an automated script and increase the chances of
        # successfully completing your automation tasks.
        options.add_experimental_option('useAutomationExtension', False)
    
        # This argument tells the browser to ignore any SSL certificate errors that may occur while accessing a website,
        # which is useful when testing on a site with a self-signed or invalid SSL certificate. Without this argument,
        # the browser will display a warning message about the certificate and require manual confirmation to proceed.
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors=yes')
    
        # wait for the page to be fully loaded before proceeding
        # options.page_load_strategy = 'normal'  # 'none', 'eager', or 'normal'
    
        prefs={
            "disable-transitions": True,
            "profile.managed_default_content_settings.images": 2,
            "profile.default_content_setting_values.notifications": 2
        }
        options.add_experimental_option("prefs", prefs)
    
        return driver
    
    
        def check_internet_connection(self):
        host = "8.8.8.8"  # Google's public DNS server
        max_attempts = 5
        interval = 10  # in seconds

        for _ in range(max_attempts):
            response_time = ping(host)
            if response_time is not None:
                # print("Internet connection is active.")
                return True
            else:
                print("No internet connection. Retrying in {} seconds...".format(interval))
                time.sleep(interval)

        print("No internet connection after multiple attempts.")
        return False

    def click_seemore(self):
        while True:
            try:
                self.wait.until(
                    EC.presence_of_element_located((By.XPATH, "//ul[@id='listAvailableStores']/following-sibling::a")))
                seemore_but = self.driver.find_element(By.XPATH, "//ul[@id='listAvailableStores']/following-sibling::a")
                time.sleep(1)
                self.driver.execute_script("arguments[0].scrollIntoView(true);", seemore_but)

                self.action.move_to_element(seemore_but).click().perform()
                time.sleep(1)

            except (ElementNotInteractableException, NoSuchElementException, TimeoutException):
                break
            except StaleElementReferenceException:
                continue

    def record(self, data_list):
        if not os.path.exists("../a_output"):
            os.makedirs(f"../a_output")

        with open(self.file_name, "a") as file:
            writer = csv.DictWriter(file,
                                    fieldnames=["ProductName", "Brand", "Channel", "Province", "StoreName",
                                                "GoogleLink", "Color", "Stock", "Promotion_Price",
                                                "Additional_Promotion", "Note", "Special_Note", "Link", "Date"],
                                    delimiter=";")
            if os.stat(self.file_name).st_size == 0:
                writer.writeheader()
            for row in data_list:
                writer.writerow(row)

    def no_internet_reload(self, link, list_province_to_check):
        # Call the function to check for internet connection before proceeding
        if self.check_internet_connection():
            pass
        else:
            print("no_internet_reload(restart_link)")
            self.check(link, list_province_to_check)

    def check(self, link, list_province_to_check):
        try:
            # Clear all cookies
            self.driver.delete_all_cookies()

            self.driver.get(link)
            time.sleep(10)

            # try:
            self.wait.until(EC.presence_of_element_located((By.XPATH, "//h1")))
            ProductName = self.driver.find_element(By.XPATH, "//h1").text

            # Stock
            try:
                special_note = self.driver.find_element(By.CLASS_NAME, "productstatus")
            except NoSuchElementException:
                try:
                    self.driver.find_element(By.CLASS_NAME, "buttonsub")  # Just for Pre-order
                except NoSuchElementException:
                    ton_kho = "Yes"
                else:
                    ton_kho = "No"
            else:
                # This case will appear in case of preorder and not trading
                if ("ngừng" or "tạm") in special_note.text.lower():
                    ton_kho = "not trading"
                else:
                    ton_kho = "No"

            if ton_kho == "Yes":
                def click_buy_but():
                    try:
                        self.wait.until(
                            EC.presence_of_element_located((By.XPATH, "//div[@class='block-button normal']/a[1]")))
                        buy_but = self.driver.find_element(By.XPATH, "//div[@class='block-button normal']/a[1]")
                        self.driver.execute_script("arguments[0].scrollIntoView(true);", buy_but)
                        self.driver.execute_script("arguments[0].click();", buy_but)
                    except ElementNotInteractableException:
                        try:
                            print("This is when they have special sales promotion")
                            buy_but = self.driver.find_element(By.XPATH, "(//div[@class='block-button twoprice']/a)[1]")
                            self.driver.execute_script("arguments[0].scrollIntoView(true);", buy_but)
                            # self.action.move_to_element(buy_but).click().perform()
                            self.driver.execute_script("arguments[0].click();", buy_but)
                        except NoSuchElementException:
                            print("This is when they have another promotion")
                            buy_but = self.driver.find_element(By.XPATH,
                                                               "(//a[@class='button medium orange inline'])[1]")
                            self.driver.execute_script("arguments[0].scrollIntoView(true);", buy_but)
                            # self.action.move_to_element(buy_but).click().perform()
                            self.driver.execute_script("arguments[0].click();", buy_but)

                    # self.wait for the confirm to add to. Notice, accessories do not have this
                    try:
                        WebDriverWait(self.driver, 15).until(
                            EC.presence_of_element_located((By.XPATH, "(//button[@class='ordered'])[1]")))
                        confirm_but = self.driver.find_element(By.XPATH, "(//button[@class='ordered'])[1]")
                        # self.action.move_to_element(confirm_but).click().perform()
                        self.driver.execute_script("arguments[0].click();", confirm_but)
                        time.sleep(5)
                    except TimeoutException:
                        pass

                def method_1():

                    # Try if the close notification appear and invisible
                    try:
                        WebDriverWait(self.driver, 15).until(
                            EC.invisibility_of_element((By.XPATH, "(//div[@class='alert-promo']/div)[1]")))
                    except TimeoutException:
                        print("(//div[@class='alert-promo']/div)[1]")
                        close_noti = self.driver.find_element(By.XPATH, "(//div[@class='alert-promo']/div)[1]")
                        self.driver.execute_script("arguments[0].scrollIntoView(true);",
                                                   close_noti)
                        self.driver.execute_script("arguments[0].click();", close_noti)
                        time.sleep(1)
                        click_buy_but()

                        try:
                            self.driver.find_element(By.XPATH,
                                                     "(//div[@class='box-promo']//i[@class='cartnew-choose'])[1]")
                        except NoSuchElementException:
                            pass
                        else:
                            choose_1_promotion=self.driver.find_element(By.XPATH,
                                                                        "(//div[@class='box-promo']//i[@class='cartnew-choose'])[1]")
                            self.driver.execute_script("arguments[0].scrollIntoView(true);", choose_1_promotion)
                            self.driver.execute_script("arguments[0].click();", choose_1_promotion)
                            time.sleep(1.5)
                            ordered_but=self.driver.find_element(By.XPATH, "(//button[@class='ordered'])[1]")
                            self.driver.execute_script("arguments[0].scrollIntoView(true);", ordered_but)
                            self.driver.execute_script("arguments[0].click();", ordered_but)
                            time.sleep(1.5)

                    else:
                        pass
                    print("Go with the method 1")

                    # try:
                    #     # in case the cart have more than 2 products, have to click to the cart
                    #     self.wait.until(
                    #         EC.presence_of_element_located((By.XPATH, "//span[@class='cart-number has-cart']")))
                    #     cart_num = self.driver.find_element(By.XPATH, "//span[@class='cart-number has-cart']")
                    #     if int(cart_num.text) > 1:
                    #         cart = self.driver.find_element((By.XPATH, "//i[@class='iconnewglobal-cart']"))
                    #         self.action.move_to_element(cart).click().perform()
                    #     # if there is more than 1 product, delete
                    #     self.remove_unwanted_product()
                    # except NoSuchElementException:
                    #     self.check(link, list_province_to_check=list_province)

                    # Choose go to physical store
                    print("# Choose go to physical store")
                    self.wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='click-choose']/div[2]/i")))
                    chose_onsite = self.driver.find_element(By.XPATH, "//div[@class='click-choose']/div[2]/i")
                    try:
                        print("try physical store")
                        # self.action.move_to_element(chose_onsite).click().perform()
                        self.driver.execute_script("arguments[0].click();", chose_onsite)
                        time.sleep(2)

                        # Try if the close notification appear and invisible
                        try:
                            WebDriverWait(self.driver, 15).until(
                                EC.invisibility_of_element((By.XPATH, "(//div[@class='alert-promo']/div)[1]")))
                        except TimeoutException:
                            print("(//div[@class='alert-promo']/div)[1]")
                            close_noti = self.driver.find_element(By.XPATH, "(//div[@class='alert-promo']/div)[1]")
                            self.driver.execute_script("arguments[0].scrollIntoView(true);",
                                                       close_noti)
                            self.driver.execute_script("arguments[0].click();", close_noti)
                            time.sleep(1)
                        else:
                            pass

                    except ElementNotInteractableException:
                        print("try physical store ElementNotInteractableException")

                    outer_loop_break = False  # Flag to control the outer loop
                    # click to the province name
                    for province in list_province_to_check:
                        # Handle internet interupted
                        self.restart_province = province
                        self.list_province_temp = list_province[list_province.index(self.restart_province):]
                        self.no_internet_reload(link, list_province_to_check=self.list_province_temp)

                        # choose province
                        max_retry = 3  # Maximum number of retries
                        retry_count = 0  # Counter for retries

                        while retry_count < max_retry:
                            # Click on the province_field to show provinces
                            province_field = self.driver.find_element(By.XPATH,
                                                                      "(//div[@class='btn-click country'])[2]/button")
                            try:
                                # note: ElementNotInteractableException
                                self.action.move_to_element(province_field).click().perform()
                                time.sleep(2)  # Wait after clicking
                                self.driver.find_element(By.XPATH, "//div[@class='select active']/div/input")
                                break  # If the element province_field_2 is found, exit the loop
                            except Exception as e:
                                # Handle any unexpected exceptions within this block
                                print("An unexpected error occurred:", e)
                                # If the element is not found, increment the retry_count and try again
                                retry_count += 1
                                time.sleep(2)  # Wait before the next attempt

                        # Check if the element was found or if the max retries were reached
                        if retry_count == max_retry:
                            print("Timed out after maximum retries at province_field_2.")
                            print("restart province:", province)
                            self.restart_province = province
                            self.list_province_temp = list_province[
                                                      list_province.index(self.restart_province):]
                            self.check(link, list_province_to_check=self.list_province_temp)
                            break
                        else:
                            # Continue with your code using province_field_2
                            self.wait.until(
                                EC.presence_of_element_located((By.XPATH, "//div[@class='select active']/div/input")))

                            province_field_2 = self.driver.find_element(By.XPATH,
                                                                        "//div[@class='select active']/div/input")
                            self.action.move_to_element(province_field_2).click().perform()
                            time.sleep(.5)
                            self.action.key_down(Keys.COMMAND).send_keys('a').key_up(Keys.COMMAND).send_keys(
                                Keys.DELETE).perform()
                            time.sleep(.5)
                            self.action.send_keys(province).perform()
                            time.sleep(.8)
                            print(province)
                        try:
                            # The matched province will be shown here
                            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located(
                                (By.XPATH, "//div[@class='select active']/div[2]/aside[1]/span")))
                            province_chosen = self.driver.find_element(By.XPATH,
                                                                       "//div[@class='select active']/div[2]/aside[1]/span")
                            self.action.move_to_element(province_chosen).click().perform()
                            time.sleep(3)
                        except TimeoutException:
                            # Some province does not show, then we pass
                            continue

                        # Choose color later
                        self.wait.until(EC.presence_of_element_located(
                            (By.XPATH, "//div[@class='product-size-and-color-selection']/aside/div/div")))
                        # color_options = self.driver.find_elements(By.XPATH,
                        #                                          "//div[@class='product-size-and-color-selection']/aside/div/div")

                        # Extract the raw HTML content from the page
                        page_source = self.driver.page_source

                        # Create a Beautiful Soup object for parsing
                        soup = BeautifulSoup(page_source, "html.parser")

                        # Find all div elements with class 'product-size-and-color-selection' and nested div elements
                        color_options = soup.select("div.product-size-and-color-selection aside div div")
                        for color in color_options:
                            color = color.get_text()
                            # Handle no internet
                            self.restart_province = province
                            self.list_province_temp = list_province[
                                                      list_province.index(self.restart_province):]
                            self.no_internet_reload(link, list_province_to_check=self.list_province_temp)

                            # If only 1 color option, not click any thing
                            if len(color_options) > 1:
                                def click_color_but():
                                    # Handle no internet
                                    self.restart_province = province
                                    self.list_province_temp = list_province[
                                                              list_province.index(self.restart_province):]
                                    self.no_internet_reload(link, list_province_to_check=self.list_province_temp)

                                    # First step, click into color option button
                                    color_but = self.driver.find_element(By.XPATH,
                                                                         "//div[@class='product-size-and-color-selection']/aside/label/span[2]")
                                    self.driver.execute_script("arguments[0].scrollIntoView(true);", color_but)
                                    # note: StaleElementReferenceException
                                    self.action.move_to_element(color_but).click().perform()
                                    time.sleep(1)

                                def click_color_option():
                                    # Handle no internet
                                    self.restart_province = province
                                    self.list_province_temp = list_province[
                                                              list_province.index(self.restart_province):]
                                    self.no_internet_reload(link, list_province_to_check=self.list_province_temp)

                                    for num in range(len(color_options)):
                                        color_chosen = self.driver.find_element(By.XPATH,
                                                                                f"(//div[@class='product-size-and-color-selection']/aside/div/div/small)[{num + 1}]")

                                        if color.strip() == color_chosen.get_attribute("textContent").strip():
                                            self.driver.execute_script("arguments[0].scrollIntoView(true);",
                                                                       color_chosen)
                                            self.action.move_to_element(color_chosen).click().perform()
                                            time.sleep(1)
                                            print("click: ", color_chosen.get_attribute("textContent"))
                                            break

                                # choose province
                                max_retry_2 = 20  # Maximum number of retries
                                retry_count_2 = 0  # Counter for retries

                                while retry_count_2 < max_retry_2:
                                    try:
                                        # click on the color button to drop the color menu
                                        click_color_but()
                                        # Second step, click into color option
                                        click_color_option()
                                        break  # If the element province_field_2 is found, exit the loop
                                    # except (ElementNotInteractableException, TimeoutException,
                                    #         StaleElementReferenceException, NoSuchElementException):
                                    #     # If the element is not found, increment the retry_count and try again
                                    #     retry_count_2 += 1
                                    #     print("retry_count_2 one: ", retry_count_2)
                                    #     time.sleep(2)  # Wait before the next attempt
                                    except Exception as e:
                                        print("An unexpected error occurred:", e)
                                        retry_count_2 += 1
                                        print("retry_count_2 two: ", retry_count_2)
                                        time.sleep(2)

                                # Check if the element was found or if the max retries were reached
                                if retry_count_2 == max_retry_2:
                                    print("Timed out after maximum retries at color_option_chosen.")
                                    print("restart province:", province)
                                    self.restart_province = province
                                    self.list_province_temp = list_province[
                                                              list_province.index(self.restart_province):]
                                    self.check(link, list_province_to_check=self.list_province_temp)
                                    outer_loop_break = True  # Set the flag to True
                                    break  # This will break the inner loop
                                else:
                                    pass

                            # In case, the page lag and there is no store show off
                            try:
                                # Wait until the store list appears
                                self.wait.until(EC.presence_of_element_located(
                                    (By.XPATH, "//ul[@id='listAvailableStores']/li")))
                            except (NoSuchElementException, TimeoutException):
                                self.restart_province = province
                                self.list_province_temp = list_province[
                                                          list_province.index(self.restart_province):]
                                self.check(link, list_province_to_check=self.list_province_temp)

                            # Extract the raw HTML content from the page
                            page_source = self.driver.page_source

                            # Create a Beautiful Soup object for parsing
                            soup = BeautifulSoup(page_source, "html.parser")

                            # Find the list of full stores using Beautiful Soup
                            stores = soup.select("ul#listAvailableStores li")

                            # Find the element with the "data-tmp-price-discount" attribute
                            promotion_price = soup.find('div', {'data-tmp-price-discount': True})
                            if promotion_price:
                                promotion_price = promotion_price['data-tmp-price-discount']
                            else:
                                promotion_price = "None"

                            try:
                                additional_promotion = soup.select_one("div.promo").get_text()
                            except TypeError:
                                additional_promotion = "None"

                            data_list = []
                            for store in stores:
                                Province = province
                                Channel = "MW"
                                # Find the color using Beautiful Soup
                                Color = soup.select_one("div.select-color.active small").get_text(strip=True)
                                Note = store.select_one("span small").get_text(strip=True)

                                # StoreName = store.find_element(By.XPATH, "./span").text.replace(Note, "").strip()
                                StoreName = store.select_one("span").get_text(strip=True).replace(Note, "")

                                # stock_status = store.find_element(By.XPATH, "./span/small").get_attribute('class')
                                stock_status = store.select_one("span small")["class"]
                                if 'cohang' in stock_status:
                                    Stock = "Yes"
                                else:
                                    Stock = "No"
                                GoogleLink = ""
                                data = {"ProductName": ProductName, "Brand": self.Brand, "Channel": Channel,
                                        "Province": Province,
                                        "StoreName": StoreName,
                                        "GoogleLink": GoogleLink, "Color": Color, "Stock": Stock, "Promotion_Price": promotion_price,
                                                "Additional_Promotion": additional_promotion, "Note": Note, "Special_Note": "None",
                                        "Link": link, "Date": now_1}
                                data_list.append(data)
                            self.record(data_list)

                        if outer_loop_break:  # Check the flag
                            print("break the province loop to avoid redundancy")
                            break  # This will break the outermost loop

                try:
                    click_buy_but()
                    time.sleep(1)
                    method_1()

                except UnexpectedAlertPresentException:

                    print("try inside the new code")
                    WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(
                        (By.XPATH, "//div[@class='district place dl-border']/strong")))
                    district_but = self.driver.find_element(
                        By.XPATH, "//div[@class='district place dl-border']/strong")
                    self.driver.execute_script("arguments[0].click();", district_but)
                    self.wait.until(
                        EC.element_to_be_clickable((By.XPATH, "(//div[@class='scroll'])[2]/aside/a")))
                    first_district = self.driver.find_element(By.XPATH, "(//div[@class='scroll'])[2]/aside/a")
                    # self.action.move_to_element(first_district).click().perform()
                    self.driver.execute_script("arguments[0].click();", first_district)
                    self.wait.until(
                        EC.element_to_be_clickable((By.XPATH, "(//div[@class='scroll'])[3]/aside/a")))
                    first_ward = self.driver.find_element(By.XPATH, "(//div[@class='scroll'])[3]/aside/a")
                    # self.action.move_to_element(first_ward).click().perform()
                    self.driver.execute_script("arguments[0].click();", first_ward)
                    time.sleep(7)

                    print("Click the buy button 1 more time")
                    click_buy_but()

                    # Continue with method_1
                    print("# Continue with method_1")
                    method_1()
            else:
                data = {"ProductName": ProductName, "Brand": self.Brand, "Channel": "MW",
                        "Province": "not trading",
                        "StoreName": "not trading",
                        "GoogleLink": "not trading", "Color": "not trading", "Stock": 0, "Note": "not trading",
                        "Special_Note": "None", "Link": link, "Date": now_1}
                data_list = [data]
                self.record(data_list)

        except ConnectionResetError:

            print("Restart from the end of the code: ConnectionResetError")

            if self.restart_province == None:

                self.list_province_temp = list_province

            else:

                self.list_province_temp = list_province[list_province.index(self.restart_province):]

            self.check(link, list_province_to_check=self.list_province_temp)

        except WebDriverException as e:

            if "ERR_CONNECTION_TIMED_OUT" in str(e):

                print("Restart from the end of the code: ERR_CONNECTION_TIMED_OUT")

                if self.restart_province == None:

                    self.list_province_temp = list_province

                else:

                    self.list_province_temp = list_province[list_province.index(self.restart_province):]

                self.check(link, list_province_to_check=self.list_province_temp)

